## Import des librairies

On commence par importer toutes les librairies

In [ ]:
import sys
sys.path.insert(1, 'aquilign')
import json
import os

import string
from numpyencoder import NumpyEncoder
import sys
import numpy as np
import random
# import collatex
import align.graph_merge as graph_merge
import align.utils as utils
import preproc.tok_apply as tokenize
import preproc.syntactic_tokenization as syntactic_tokenization
from align.encoder import Encoder
from align.aligner import Bertalign
import pandas as pd
import argparse
import glob

On vérifie que le code de l'aligneur est bien importé:

In [ ]:
print(dir(tokenize))

## Fonction d'alignement

Cela semble marcher. Produisons la fonction globale qui permet de gérer le processus entier d'alignement:

In [ ]:
class Aligner:
    """
    La classe Aligner initialise le moteur d'alignement, fondé sur Bertalign
    """

    def __init__(self,
                 model,
                 corpus_limit:None, 
                 max_align=3, 
                 out_dir="out", 
                 use_punctuation=True, 
                 input_dir="in", 
                 main_wit=None, 
                 prefix=None,
                 device="cpu",
                 tokenizer="regexp", 
                 tok_models=None
                 ):
        self.model = model
        self.alignment_dict = dict()
        self.text_dict = dict()
        self.files_path = glob.glob(f"{input_dir}/*/*.txt")
        self.device = device
        assert any([main_wit in path for path in self.files_path]), "Main wit doesn't match witnesses paths, please check arguments. " \
                                                                    f"Main wit: {main_wit}, other wits: {self.files_path}"
        print(self.files_path)
        self.main_file_index = next(index for index, path in enumerate(self.files_path) if main_wit in path)
        self.corpus_limit = corpus_limit
        self.max_align = max_align
        self.out_dir = out_dir
        self.use_punctiation = use_punctuation
        self.prefix = prefix
        self.tokenizer = tokenizer
        self.tok_models = tok_models
        self.wit_pairs = self.create_pairs(self.files_path, self.main_file_index)

        try:
            os.mkdir(f"result_dir")
        except FileExistsError:
            pass
        try:
            os.mkdir(f"result_dir/{self.out_dir}/")
        except FileExistsError:
            pass
        
        # Let's check the paths are correct
        for file in self.files_path:
            assert os.path.isfile(file), f"Vérifier le chemin: {file}"
            

    def parallel_align(self):
        """
        This function procedes to the alignments two by two and then merges the alignments into a single alignement
        """
        pivot_text = self.wit_pairs[0][0]
        pivot_text_lang = pivot_text.split("/")[-2]

        # On commence par le premier texte, qui sera notre pivot
        if self.tokenizer is None:
            pass
        elif self.tokenizer == "regexp":
            first_tokenized_text = utils.clean_tokenized_content(
                syntactic_tokenization.syntactic_tokenization(input_file=pivot_text, 
                                                              corpus_limit=self.corpus_limit,
                                                              use_punctuation=True,
                                                              lang=pivot_text_lang))
        else:
            first_tokenized_text = tokenize.tokenize_text(input_file=pivot_text, 
                                                          corpus_limit=self.corpus_limit, 
                                                          remove_punct=False, 
                                                          tok_models=self.tok_models, 
                                                          output_dir=self.out_dir, 
                                                          device=self.device,
                                                          lang=pivot_text_lang)
        
        assert first_tokenized_text != [], "Erreur avec le texte tokénisé du témoin base"
        
        main_wit_name = self.wit_pairs[0][0].split("/")[-1].split(".")[0]
        utils.write_json(f"result_dir/{self.out_dir}/tokenized_{main_wit_name}.json", first_tokenized_text)
        utils.write_tokenized_text(f"result_dir/{self.out_dir}/tokenized_{main_wit_name}.txt", first_tokenized_text)
        
        # We randomize the pairs. It can help resolving memory issue.
        random.shuffle(self.wit_pairs)
        # Puis on boucle sur chacun des autres textes
        for index, (main_wit, wit_to_compare) in enumerate(self.wit_pairs):
            main_wit_name = main_wit.split("/")[-1].split(".")[0]
            wit_to_compare_name = wit_to_compare.split("/")[-1].split(".")[0]
            current_wit_lang = wit_to_compare.split("/")[-2]
            print(len(first_tokenized_text))
            if self.tokenizer is None:
                pass
            elif self.tokenizer == "regexp":
                second_tokenized_text = utils.clean_tokenized_content(
                    syntactic_tokenization.syntactic_tokenization(input_file=wit_to_compare, 
                                                                  corpus_limit=self.corpus_limit,
                                                                  use_punctuation=True, 
                                                                  lang=current_wit_lang))
            else:
                second_tokenized_text = tokenize.tokenize_text(input_file=wit_to_compare, 
                                                               corpus_limit=self.corpus_limit,
                                                               remove_punct=False, 
                                                               tok_models=self.tok_models,
                                                               output_dir=self.out_dir, 
                                                               device=self.device,
                                                               lang=current_wit_lang)
            assert second_tokenized_text != [], f"Erreur avec le texte tokénisé du témoin comparé {wit_to_compare_name}"
            utils.write_json(f"result_dir/{self.out_dir}/tokenized_{wit_to_compare_name}.json", second_tokenized_text)
            utils.write_tokenized_text(f"result_dir/{self.out_dir}/tokenized_{wit_to_compare_name}.txt", second_tokenized_text)
            
            # This dict will be used to create the alignment table in csv format
            self.text_dict[0] = first_tokenized_text
            self.text_dict[index + 1] = second_tokenized_text
            
            # Let's align the texts
            print(f"Aligning {main_wit} with {wit_to_compare}")
            
            # Tests de profil et de paramètres
            profile = 0
            if profile == 0:
                margin = True
                len_penality = True
            else:
                margin = False
                len_penality = True
            aligner = Bertalign(self.model,
                                first_tokenized_text, 
                                second_tokenized_text, 
                                max_align= self.max_align, 
                                win=5, skip=-.2, 
                                margin=margin, 
                                len_penalty=len_penality, 
                                device=self.device)
            aligner.align_sents()
            
            # We append the result to the alignment dictionnary
            self.alignment_dict[index] = aligner.result
            utils.write_json(f"result_dir/{self.out_dir}/alignment_{str(index)}.json", aligner.result)
            utils.save_alignment_results(aligner.result, first_tokenized_text, second_tokenized_text,
                                         f"{main_wit_name}_{wit_to_compare_name}", self.out_dir)
        print("Done !")
        utils.write_json(f"result_dir/{self.out_dir}/alignment_dict.json", self.alignment_dict)

    def create_pairs(self, full_list:list, main_wit_index:int) -> list[tuple]:
        """
        From a list of witnesses and the main witness index, create all possible pairs with this witness. Returns a list 
        of tuples with the main wit and the wit to compare it to
        """
        pairs = []
        main_wit = full_list.pop(int(main_wit_index))
        for wit in full_list:
            pairs.append((main_wit, wit))
        return pairs

    def save_final_result(self, merged_alignments:list, delimiter="\t"):
        """
        Saves result to csv file
        """
        
        all_wits = [self.wit_pairs[0][0]] + [pair[1] for pair in self.wit_pairs]
        filenames = [wit.split("/")[-1].replace(".txt", "") for wit in all_wits]
        with open(f"result_dir/{self.out_dir}/final_result.csv", "w") as output_text:
            output_text.write(delimiter + delimiter.join(filenames) + "\n")
            # TODO: remplacer ça, c'est pas propre et ça sert à rien
            translation_table = {letter:index for index, letter in enumerate(string.ascii_lowercase)}
            for alignment_unit in merged_alignments:
                output_text.write("|".join(value for value in alignment_unit['a']) + delimiter)
                for index, witness in enumerate(merged_alignments[0]):
                    output_text.write("|".join(self.text_dict[translation_table[witness]][int(value)] for value in
                                               alignment_unit[witness]))
                    if index + 1 != len(merged_alignments[0]):
                        output_text.write(delimiter)
                output_text.write("\n")
        
        
        with open(f"result_dir/{self.out_dir}/readable.csv", "w") as output_text:
            output_text.write(delimiter.join(filenames) + "\n")
            # TODO: remplacer ça, c'est pas propre et ça sert à rien
            translation_table = {letter:index for index, letter in enumerate(string.ascii_lowercase)}
            for alignment_unit in merged_alignments:
                for index, witness in enumerate(merged_alignments[0]):
                    output_text.write(" ".join(self.text_dict[translation_table[witness]][int(value)] for value in
                                               alignment_unit[witness]))
                    if index + 1 != len(merged_alignments[0]):
                        output_text.write(delimiter)
                output_text.write("\n")
        
        with open(f"result_dir/{self.out_dir}/final_result_as_index.csv", "w") as output_text:
            output_text.write(delimiter + delimiter.join(filenames) + "\n")
            for alignment_unit in merged_alignments:
                for index, witness in enumerate(merged_alignments[0]):
                    output_text.write("|".join(value for value in
                                               alignment_unit[witness]))
                    if index + 1 != len(merged_alignments[0]):
                        output_text.write(delimiter)
                output_text.write("\n")

        data = pd.read_csv(f"result_dir/{self.out_dir}/final_result.csv", delimiter="\t")
        # Convert the DataFrame to an HTML table
        html_table = data.to_html()
        full_html_file = f"""<html>
                          <head>
                          <title>Alignement final</title>
                            <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
                            </head>
                          <body>
                          {html_table}
                          </body>
                    </html>"""
        with open(f"result_dir/{self.out_dir}/final_result.html", "w") as output_html:
            output_html.write(full_html_file)

## Paramètres

On va enfin donner les différents paramètres à l'outil pour aligner nos textes. En utilisant l'interface de ligne de commande (CLI), on aurait: `python3 main.py -o lancelot -i data/extraitsLancelot/ii-48/ -mw data/extraitsLancelot/ii-48_extrait/fr/micha-ii-48.txt -d 
cuda:0 -t bert-based`. Il faut ici directement renseigner les différents arguments (dossier de sortie, dossier d'entrée, témoin-pivot, préfixe des fichiers à produire, instrument de calcul, type de segmenteur, modèles de segmentation, utilisation de la ponctuation dans l'alignement.

In [ ]:
out_dir = "lancelot"
input_dir = "data/extraitsLancelot/ii-48_extrait"
main_wit = "data/extraitsLancelot/ii-48_extrait/fr/micha-ii-48.txt"
assert input_dir != None,  "Input dir is mandatory"
assert main_wit != None,  "Main wit path is mandatory"
prefix = ""

On renseignera `cuda:0` si une carte graphique est à disposition, ce qui permet d'accélerer le traitement. Binder ne permet pas d'utiliser de carte graphique et il faut alors indiquer `cpu`.

In [ ]:
device = "cuda:0"
# device = "cpu"

In [ ]:
corpus_limit = None
if corpus_limit:
    corpus_limit = float(corpus_limit)
tokenizer = "bert-based"

Les modèles de segmentation sont publiés sur HuggingFace, et sont directement disponibles à l'aide de la librairie Transformers.

In [ ]:
tok_models = {"fr": 
                  {"model": "ProMeText/aquilign_french_segmenter", 
                   "tokenizer": "dbmdz/bert-base-french-europeana-cased", 
                   "tokens_per_example": 12}, 
              "es": {"model": "ProMeText/aquilign_spanish_segmenter", 
                     "tokenizer": "dccuchile/bert-base-spanish-wwm-cased", 
                     "tokens_per_example": 30}, 
              "it": {"model": "ProMeText/aquilign_italian_segmenter", 
                     "tokenizer": "dbmdz/bert-base-italian-xxl-cased", 
                     "tokens_per_example": 12}, 
              "la": {"model": "ProMeText/aquilign_segmenter_latin", 
                     "tokenizer": "LuisAVasquez/simple-latin-bert-uncased", 
                     "tokens_per_example": 50}}
assert tokenizer in ["None", "regexp", "bert-based"], "Authorized values for tokenizer are: None, regexp, bert-based"
if tokenizer == "None":
    tokenizer = None
use_punctuation = False

On peut maintenant lancer l'outil !

## Lancement de l'alignement

On choisit d'abord le modèle de `sentence embeddings`.

In [ ]:
# Initialize model 
models = {0: "distiluse-base-multilingual-cased-v2", 1: "LaBSE", 2: "Sonar"}
model = Encoder(models[int(1)], device=device)

Dans un deuxième temps, on initialise une instance de l'aligneur, qui imprime toutes les paires d'alignements qui vont être produites.

In [ ]:
print(f"Punctuation for tokenization: {use_punctuation}")
MyAligner = Aligner(model, corpus_limit=corpus_limit, 
                    max_align=3, 
                    out_dir=out_dir, 
                    use_punctuation=use_punctuation, 
                    input_dir=input_dir, 
                    main_wit=main_wit, 
                    prefix=prefix, 
                    device=device, 
                    tokenizer=tokenizer, 
                    tok_models=tok_models)

On lance l'alignement parallèle: il y aura autant de résultats d'alignements que de paires de témoins. La fonction va commencer par installer les différents modèles si le notebook est lancé pour la première fois.

In [ ]:
MyAligner.parallel_align()

La classe `Aligner` produit un dictionnaire qui recense l'ensemble des alignements par paire. Il va s'agir d'enregistrer ce dictionnaire dans un fichier json, `alignment_dict.json`.


In [ ]:
utils.write_json(f"result_dir/{out_dir}/alignment_dict.json", MyAligner.alignment_dict)
align_dict = utils.read_json(f"result_dir/{out_dir}/alignment_dict.json")

L'étape suivante est celle de la fusion des tables d'alignement individuelles en une seule table.

In [ ]:
list_of_merged_alignments = graph_merge.merge_alignment_table(align_dict)

### Tests
On teste les résultats

In [ ]:
# On teste si on ne perd pas de noeuds textuels
print("Testing results consistency")
possible_witnesses = string.ascii_lowercase[:len(align_dict) + 1]
tested_table = utils.test_tables_consistency(list_of_merged_alignments, possible_witnesses)
# TODO: une phase de test pour voir si l'alignement final est cohérent avec les alignements deux à deux

### Production des fichiers de sortie
On enregistre les fichiers et on produit le document HTML.

In [ ]:
# Let's save the final tables (indices and texts)
MyAligner.save_final_result(merged_alignments=list_of_merged_alignments)

Le résultat se trouve dans [result_dir/lancelot/final_result.html](result_dir/lancelot/final_result.html). 